# Lector de matricules
#### v1
L'objectiu d'aquest notebook es fer un proof of concept de la pipeline.

Aquesta pipeline s'encarregara de fer:
* Llegir imatges
* Aplicar CLAHE per normalitzar valors
* Aplicar thresholding (binary, otsu) per convertir-la en B/N
* Seleccionar regio de la matricula
* Fer un crop de la ROI i aplicar homografia per alinear la geometria
* Passat OCR
* Desar valors

In [1]:
import cv2